In [4]:
import api


In [5]:
config = api.CloudberryConfig('http://localhost', 9000)
logs_api = api.LogsApi(config)
flux_api = api.FluxApi(config)

In [6]:
evaluation_id = "3e29294d-bf7c-43c6-86b8-c9073de2685f"
workplace_id = 0

logs_query = logs_api \
    .workplace() \
    .by_evaluation_id(evaluation_id) \
    # .by_workplace_id(workplace_id)

logs = logs_query.get_raw()
logs_df = logs_query.get_dataframe()

In [7]:
logs

[{'time': {'nano': 758000000, 'epochSecond': 1592503095},
  'evaluationId': '3e29294d-bf7c-43c6-86b8-c9073de2685f',
  'workplaceId': 0,
  'parameters': {}}]

In [8]:
logs_df

,time,evaluationId,workplaceId,parameters
0,"{'nano': 758000000, 'epochSecond': 1592503095}",3e29294d-bf7c-43c6-86b8-c9073de2685f,0,{}


In [9]:
bucket_name = "cloudberry-logs"
raw_query = f"from(bucket:{bucket_name}) |> range(start:0) |> filter(fn: (r) => r.workplaceId == 0)"
result = flux_api.query(raw_query)

In [ ]:
result